In [15]:
# 1. 라이브러리 불러오기
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# 2. 데이터 로드 및 확인
titanic = sns.load_dataset('titanic')

# 2-3. describe 함수 설명
# count: 데이터의 개수, nan이 아닌 값의 개수를 보여줍니다. 
# std: 데이터의 표준 편차로, 데이터가 평균에서 얼마나 퍼져있는지를 나타냅니다.
# min: 데이터의 최솟값
# 25%: 1사분위수, 데이터의 25%가 이 값 이하임을 나타냅니다.
# 50%: 중앙값, 데이터의 중간값을 나타냅니다.
# 75%: 3사분위수, 데이터의 75%가 이 값 이하임을 나타냅니다.
# max: 데이터의 최댓값

# 3-1. 결측치 처리
titanic['age'] = titanic['age'].fillna(titanic['age'].median())  # inplace=True 대신 직접 할당
titanic['embarked'] = titanic['embarked'].fillna(titanic['embarked'].mode()[0])  # inplace=True 대신 직접 할당

# 3-2. 수치형으로 인코딩
titanic['sex'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic['alive'] = titanic['alive'].map({'yes': 1, 'no': 0})
titanic['embarked'] = titanic['embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# 3-3. 새로운 feature 생성
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

# 4-1. 모델 학습 준비
titanic = titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']]
X = titanic.drop('survived', axis=1)  # feature
y = titanic['survived']  # target

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. 모델 학습 및 평가
# 5-1. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 5-2. XGBoost 모델 학습
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

# 5-3. 예측 및 평가
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'XGBoost 정확도: {accuracy:.4f}')
print(f'XGBoost 평균 제곱 오차: {mse:.4f}')

XGBoost 정확도: 0.7933
XGBoost 평균 제곱 오차: 0.2067


In [16]:
# 5-2. XGBoost 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(XGBClassifier(random_state=42), param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# 최적의 파라미터로 모델 재학습
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 5-3. 평가
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'최적의 파라미터: {grid_search.best_params_}')
print(f'XGBoost 정확도: {accuracy:.4f}')
print(f'XGBoost 평균 제곱 오차: {mse:.4f}')

최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
XGBoost 정확도: 0.8268
XGBoost 평균 제곱 오차: 0.1732


In [17]:
# 5-2. XGBoost 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]  # 새로운 하이퍼파라미터 추가
}

grid_search = GridSearchCV(XGBClassifier(random_state=42), param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# 최적의 파라미터로 모델 재학습
best_model = grid_search.best_estimator_

# 5-3. 앙상블 모델 구축
rf = RandomForestClassifier(n_estimators=100, random_state=42)
voting_clf = VotingClassifier(estimators=[('xgb', best_model), ('rf', rf)], voting='soft')
voting_clf.fit(X_train, y_train)

# 5-4. 예측 및 평가
y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'최적의 파라미터: {grid_search.best_params_}')
print(f'앙상블 모델 정확도: {accuracy:.4f}')
print(f'앙상블 모델 평균 제곱 오차: {mse:.4f}')

최적의 파라미터: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1.0}
앙상블 모델 정확도: 0.8268
앙상블 모델 평균 제곱 오차: 0.1732


In [20]:
# 1. 라이브러리 불러오기
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 데이터 로드 및 확인
titanic = sns.load_dataset('titanic')

# 결측치 확인
print(titanic.isnull().sum())

# 결측치 처리
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['fare'] = titanic['fare'].fillna(titanic['fare'].median())
titanic['embarked'] = titanic['embarked'].fillna(titanic['embarked'].mode()[0])

# 수치형으로 인코딩
titanic['sex'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic['alive'] = titanic['alive'].map({'yes': 1, 'no': 0})
titanic['embarked'] = titanic['embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# 새로운 feature 생성
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

# 모델 학습 준비
X = titanic[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']]
y = titanic['survived']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. 모델 학습 및 평가
# 5-1. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 5-2. XGBoost 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [300, 400],
    'max_depth': [4, 5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

grid_search = GridSearchCV(XGBClassifier(random_state=42), param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# 최적의 파라미터로 모델 재학습
best_model = grid_search.best_estimator_

# 5-3. 앙상블 모델 구축
rf = RandomForestClassifier(n_estimators=150, random_state=42)
voting_clf = VotingClassifier(estimators=[('xgb', best_model), ('rf', rf)], voting='soft')
voting_clf.fit(X_train, y_train)

# 5-4. 예측 및 평가
y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'최적의 파라미터: {grid_search.best_params_}')
print(f'앙상블 모델 정확도: {accuracy:.4f}')
print(f'앙상블 모델 평균 제곱 오차: {mse:.4f}')


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
최적의 파라미터: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.7}
앙상블 모델 정확도: 0.8324
앙상블 모델 평균 제곱 오차: 0.1676
